In [19]:
import requests
import pandas as pd
import time
import os
import re

In [11]:
os.makedirs('data/', exist_ok=True)

I tried scraping but it didn't work. Instead you can get it from an API!

In [12]:
import requests

#START HERE

In [14]:
import requests

headers = {
    'accept': '*/*',
    'accept-language': 'en',
    'access-control-allow-credentials': 'true',
    'access-control-allow-methods': '*',
    'access-control-allow-origin': '*',
    'content-type': 'application/json',
    'origin': 'https://nypdonline.org',
    'priority': 'u=1, i',
    'referer': 'https://nypdonline.org/link/1016',
    'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
}

json_data = [
    {
        'key': '@DocumentKeyword',
        'values': [
            '',
        ],
    },
]

response = requests.post(
    'https://nypdonline.org/api/reports/ed551b4a-cd5c-4d8e-bcb9-a0478b4c5dea/data',
    headers=headers,
    json=json_data,
)

data = response.json()
df = pd.DataFrame(data)
df.to_csv("nypd_trial_decisions.csv", index=False)

In [20]:
pd.json_normalize(data)
df = pd.DataFrame(data)

this uses an undocumented API and then cURL converter, then converst the json to a dataframe

In [21]:
df.info

<bound method DataFrame.info of      filterKey                                            columns
0       @TAXID  [{'value': '2025'}, {'value': 'July'}, {'value...
1       @TAXID  [{'value': '2025'}, {'value': 'June'}, {'value...
2       @TAXID  [{'value': '2025'}, {'value': 'June'}, {'value...
3       @TAXID  [{'value': '2025'}, {'value': 'May'}, {'value'...
4       @TAXID  [{'value': '2025'}, {'value': 'May'}, {'value'...
...        ...                                                ...
1730    @TAXID  [{'value': '2008'}, {'value': 'February'}, {'v...
1731    @TAXID  [{'value': '2008'}, {'value': 'February'}, {'v...
1732    @TAXID  [{'value': '2008'}, {'value': 'January'}, {'va...
1733    @TAXID  [{'value': '2008'}, {'value': 'January'}, {'va...
1734    @TAXID  [{'value': '2008'}, {'value': 'January'}, {'va...

[1735 rows x 2 columns]>

In [22]:
df.tail()

,filterKey,columns
1730,@TAXID,"[{'value': '2008'}, {'value': 'February'}, {'v..."
1731,@TAXID,"[{'value': '2008'}, {'value': 'February'}, {'v..."
1732,@TAXID,"[{'value': '2008'}, {'value': 'January'}, {'va..."
1733,@TAXID,"[{'value': '2008'}, {'value': 'January'}, {'va..."
1734,@TAXID,"[{'value': '2008'}, {'value': 'January'}, {'va..."


In [24]:
def parse_row(row):
    values = [item['value'] for item in row]
    # Extract officer name from HTML
    officer_match = re.search(r'>(.*?)<', values[3])
    officer = officer_match.group(1).strip() if officer_match else values[3]
    # Extract filename and URL from HTML
    pdf_match = re.search(r'href="([^"]+)".*?>([^<]+)</a>', values[4])
    pdf_url = f"https://nypdonline.org{pdf_match.group(1)}" if pdf_match else None
    pdf_filename = pdf_match.group(2) if pdf_match else None

    return {
        'Year': values[0],
        'Month': values[1],
        'Decision Date': values[2],
        'Officer Name': officer,
        'PDF Filename': pdf_filename,
        'PDF URL': pdf_url
    }

# Apply parsing to each row
parsed_rows = df['columns'].apply(parse_row)
df_clean = pd.DataFrame(parsed_rows.tolist())

print(df_clean.head())

   Year Month          Decision Date      Officer Name PDF Filename  \
0  2025  July   7/3/2025 12:00:00 AM  RIVERA, JONATHAN  2025021.pdf   
1  2025  June  6/27/2025 12:00:00 AM     ROPER, KAMALA  2025028.pdf   
2  2025  June  6/18/2025 12:00:00 AM       WARD, AISHA  2025027.pdf   
3  2025   May  5/30/2025 12:00:00 AM    NUNEZ, OSVALDO  2025020.pdf   
4  2025   May  5/23/2025 12:00:00 AM   BAGAN, WOJCIECH  2025026.pdf   

                                             PDF URL  
0  https://nypdonline.org/files/949550_07032025_2...  
1  https://nypdonline.org/files/926029_06272025_2...  
2  https://nypdonline.org/files/939673_06182025_2...  
3  https://nypdonline.org/files/965374_05302025_2...  
4  https://nypdonline.org/files/948637_05232025_2...  


In [26]:
df_clean.info

<bound method DataFrame.info of       Year     Month          Decision Date        Officer Name PDF Filename  \
0     2025      July   7/3/2025 12:00:00 AM    RIVERA, JONATHAN  2025021.pdf   
1     2025      June  6/27/2025 12:00:00 AM       ROPER, KAMALA  2025028.pdf   
2     2025      June  6/18/2025 12:00:00 AM         WARD, AISHA  2025027.pdf   
3     2025       May  5/30/2025 12:00:00 AM      NUNEZ, OSVALDO  2025020.pdf   
4     2025       May  5/23/2025 12:00:00 AM     BAGAN, WOJCIECH  2025026.pdf   
...    ...       ...                    ...                 ...          ...   
1730  2008  February  2/11/2008 12:00:00 AM      MEJIA, EDILIO*  2008047.pdf   
1731  2008  February   2/4/2008 12:00:00 AM      SANCHEZ, LUIS*  2008036.pdf   
1732  2008   January  1/28/2008 12:00:00 AM       MARTE, MIGUEL  2008031.pdf   
1733  2008   January  1/27/2008 12:00:00 AM    LESANE, SOLOMON*  2008044.pdf   
1734  2008   January  1/23/2008 12:00:00 AM   ALEXANDER, BRIAN*  2008057.pdf   

       

In [29]:
df_clean.to_csv("nypd_trial_decisions_cleaned.csv", index=False)

In [30]:
import requests
import os

os.makedirs("nypd_pdfs", exist_ok=True)

for i, row in df_clean.iterrows():
    url = row['PDF URL']
    filename = row['PDF Filename']

    if url and filename:
        response = requests.get(url)
        if response.status_code == 200:
            with open(os.path.join("nypd_pdfs", filename), 'wb') as f:
                f.write(response.content)